# Train on Davidson

## Import stuff

In [2]:
import os
import sys
import ipdb
from tqdm import tqdm
import numpy as np
import torch.optim as optim
from torchtext.data import TabularDataset, BucketIterator, Field
sys.path.extend(['/Users/zeerakw/Documents/PhD/projects/active/Generalisable_abuse'])

import gen.shared.custom_types as t
from gen.neural import LSTMClassifier
from gen.shared.data import BatchExtractor
from gen.shared.clean import Cleaner
#from gen.shared.train import compute_unigram_liwc

## Setup variables

In [3]:
text_label = Field(sequential = True,
                   include_lengths = False,
                   use_vocab = True,
                   pad_token = "<PAD>",
                   unk_token = "<UNK>")

int_label = Field(sequential = False,
                  include_lengths = False,
                  use_vocab = False,
                  pad_token = None,
                  unk_token = None)

In [4]:
device = 'cpu'
data_dir = '/Users/zeerakw/Documents/PhD/projects/active/Generalisable_abuse/data/'
data_file = 'davidson_test.csv'
path = os.path.join(data_dir, data_file)
file_format = 'csv'
cleaners = ['lower', 'url', 'hashtag', 'username']
clean = Cleaner(cleaners)

# Set fields
text_field = text_label
label_field = int_label

# Update training field
#setattr(text_field, 'tokenize', clean.tokenize)
#setattr(text_field, 'preprocessing', compute_unigram_liwc)
fields = [('', None), ('CF_count', None), ('hate_speech', None), ('offensive', None), ('neither', None),
          ('label', label_field), ('text', text_field)]

## Load the data

In [5]:
data = TabularDataset(path, format = file_format, fields = fields, skip_header = True)
train, test = data.split(split_ratio = 0.8, stratified = True)
loaded = (train, test)
text_field.build_vocab(data)

In [6]:
print(len(text_field.vocab))
for doc in train:
    print(len(doc.text), doc.text)
    unks = [tok for tok in doc.text if tok not in text_field.vocab.stoi]
    print(any(unks))
    res = any(tok for tok in doc.text if tok in text_field.vocab.stoi)
    print(any(tok for tok in doc.text if tok not in text_field.vocab.stoi), res)
    res = all(tok for tok in doc.text if tok not in text_field.vocab.stoi)
    print(all(tok for tok in doc.text if tok in text_field.vocab.stoi), res)
    break

3769
3 ['slut', 'whore', 'hoe']
False
False True
True True


In [7]:
batch_sizes = (2, 4)
tmp_train, tmp_test = BucketIterator.splits(loaded, batch_sizes = batch_sizes, sort_key = lambda x: len(x.text),
                                            device = device, shuffle = True, repeat = False)

In [8]:
train_batches = BatchExtractor('text', 'label', tmp_train)
test_batches = BatchExtractor('text', 'label', tmp_test)
next(iter(train_batches))

> /Users/zeerakw/Documents/PhD/projects/active/Generalisable_abuse/gen/shared/data.py(42)__iter__()
-> X = [getattr(doc, self.df) for doc in batch]
(Pdb) doc
*** NameError: name 'doc' is not defined
(Pdb) ll
 39  	    def __iter__(self):
 40  	        for batch in self.data:
 41  	            pdb.set_trace()
 42  ->	            X = [getattr(doc, self.df) for doc in batch]
 43  	            seq_len = max(map(len, X))
 44  	            y = [getattr(doc, self.lf) for doc in batch]
 45  	            yield (X, y)
(Pdb) batch

[torchtext.data.batch.Batch of size 2]
	[.label]:[torch.LongTensor of size 2]
	[.text]:[torch.LongTensor of size 23x2]
(Pdb) batch.text
tensor([[   9,   37],
        [  86,   91],
        [ 118,  242],
        [  37,  758],
        [2561,  991],
        [  14,   91],
        [ 143,  722],
        [ 155,   14],
        [ 654,  227],
        [ 750,  120],
        [   9,   81],
        [ 547,    1],
        [ 224,    1],
        [ 182,    1],
        [  26,    1],
       

AttributeError: 'tuple' object has no attribute 'text'

Internal StopIteration: True
> /Users/zeerakw/.virtualenvs/generalisable/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3242)run_ast_nodes()
-> if (await self.run_code(code, result,  async_=asy)):
(Pdb) quit


BdbQuit: 

## Define our model

In [9]:
import torch.nn as nn

In [10]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, no_classes, no_layers):
        super(LSTMClassifier, self).__init__()
        self.emb = nn.Embedding(input_dim, embedding_dim)
        self.linear = nn.Linear(embedding_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.to_output = nn.Linear(hidden_dim, no_classes)
        self.softmax = nn.LogSoftmax(dim = 1)
    
    def forward(self, sequence):
        out = self.emb(sequence)
        out, last = self.lstm(out)
        out = self.to_output(out)
        out = self.softmax(out)
        return out

In [11]:
model = LSTMClassifier(len(text_field.vocab), embedding_dim = 64, hidden_dim = 128, no_classes = 3, no_layers = 1)
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss = nn.NLLLoss()

def train(model, epochs, batches, loss_func, optimizer):
    for epoch in range(epochs):
        epoch_loss = []
        model.zero_grad()
        for X, y in batches:
            scores = model(X)
            print(X[0])
            print(X.shape, y.shape)
            loss = loss_func(scores, y)
            epoch_loss.append(float(loss))
            loss.backward()
            optimizer.step()
            break
        break
        print("Epoch {0}: Mean Loss: {1}".format(epoch, np.mean(epoch_loss)))

In [12]:
train(model, 10, train_batches, loss, optimizer)

tensor([289,   2, 271,   2,   2, 291,   2, 263, 282, 239, 292, 313, 284, 285,
          2, 129, 253, 235,   2, 128])
torch.Size([24, 20]) torch.Size([20])


ValueError: Expected input batch_size (24) to match target batch_size (20).